<a href="https://colab.research.google.com/github/nssn96/ML_GestureRecognition/blob/main/ML_HMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 4 - Gesture Recognition using HMM(Hidden Markov models)


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [99]:
#install hmmlearn using pip3
!pip install hmmlearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import pickle
from hmmlearn import hmm

In [101]:
#References Used
#https://hmmlearn.readthedocs.io/en/stable/tutorial.html#available-models
#https://datagy.io/pandas-groupby/

In [102]:
#Importing the dataset
dataset = pd.read_csv('Original_Dataset.csv')
print(dataset)

     LC_X_1  LC_X_2  LC_X_3  LC_X_4  LC_X_5  LC_X_6  LC_X_7  LC_X_8  LC_X_9  \
0       307     389     259     259     259     259     259     258     258   
1       340     340     340     340     339     339     340     339     338   
2       260     263     266     266     270     271     271     271     271   
3       338     339     338     337     337     337     337     337     336   
4       319     240     240     318     241     318     318     318     318   
..      ...     ...     ...     ...     ...     ...     ...     ...     ...   
345       0     309     309     309     309     309     309     309     309   
346     301     300     302     302     302     301     299     300     301   
347     298     295     294     292     292     290     290     288     287   
348     278     278     278     277     278     277     276     276     276   
349     279     281     281     280     280     280     280     280     281   

     LC_X_10  ...  RD_22  RD_23  RD_24  RD_25  RD_2

In [103]:
dataset.columns

Index(['LC_X_1', 'LC_X_2', 'LC_X_3', 'LC_X_4', 'LC_X_5', 'LC_X_6', 'LC_X_7',
       'LC_X_8', 'LC_X_9', 'LC_X_10',
       ...
       'RD_22', 'RD_23', 'RD_24', 'RD_25', 'RD_26', 'RD_27', 'RD_28', 'RD_29',
       'RD_30', 'Gesture'],
      dtype='object', length=781)

In [104]:
''' Since we may not need all of that information and will probably perform well
using only the hand centroids, we will be dropping some of the columns from the dataset. We will take only take the x and y values
of the hands. '''
#Reference Used
#https://datagy.io/pandas-groupby/
#Normalize each sequence and dropping some column data
def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

dropped_columns = []
for column in dataset.columns:
    if str(column).find('LF') != -1 or  str(column).find('RF') != -1 or\
            str(column).find('LD') != -1 or str(column).find('RD') != -1:
        dropped_columns.append(column)
    elif column != 'Gesture':
        dataset[column] = normalize(dataset[column])
dataset = dataset.drop(dropped_columns,axis=1)
gestures = set(dataset.Gesture)
#print(dataset)
print(gestures)   ##this is of set type

{'MoveLeft', 'MoveRight', 'MoveDown', 'ZoomOut', 'Press', 'MoveUp', 'ZoomIn'}


In [105]:
# splitting the datset into train and test set--> I am splitting 90% train and 10% test set
basic_template = [0,0,0,0,0,0,0]  # here the size is 7 because, we have seven different classes
x_train = basic_template.copy()
x_test = basic_template.copy()
y_train = basic_template.copy()
y_test = basic_template.copy()
groupby_gesture = dataset.groupby(dataset.Gesture)
print(type(groupby_gesture))
for i, gesture in enumerate(gestures):
  gesture_data = groupby_gesture.get_group(gesture)
  x_train[i],x_test[i],y_train[i],y_test[i] = train_test_split(gesture_data.iloc[: , :120],gesture_data['Gesture'],test_size=0.2, random_state=1,stratify = gesture_data['Gesture'])
print('shape of train data:', np.array(x_train).shape)
print('shape of test data:',np.array(x_test).shape )

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>
shape of train data: (7, 40, 120)
shape of test data: (7, 10, 120)


In [106]:
np.array(x_test).shape[1]

10

In [107]:
print(dataset)

       LC_X_1    LC_X_2    LC_X_3    LC_X_4    LC_X_5    LC_X_6    LC_X_7  \
0    0.647679  0.818947  0.546414  0.545263  0.546414  0.545263  0.545263   
1    0.717300  0.715789  0.717300  0.715789  0.715190  0.713684  0.715789   
2    0.548523  0.553684  0.561181  0.560000  0.569620  0.570526  0.570526   
3    0.713080  0.713684  0.713080  0.709474  0.710970  0.709474  0.709474   
4    0.672996  0.505263  0.506329  0.669474  0.508439  0.669474  0.669474   
..        ...       ...       ...       ...       ...       ...       ...   
345  0.000000  0.650526  0.651899  0.650526  0.651899  0.650526  0.650526   
346  0.635021  0.631579  0.637131  0.635789  0.637131  0.633684  0.629474   
347  0.628692  0.621053  0.620253  0.614737  0.616034  0.610526  0.610526   
348  0.586498  0.585263  0.586498  0.583158  0.586498  0.583158  0.581053   
349  0.588608  0.591579  0.592827  0.589474  0.590717  0.589474  0.589474   

       LC_X_8    LC_X_9   LC_X_10  ...   RC_Y_22   RC_Y_23   RC_Y_24  \
0  

In [108]:
#Reference
#https://hmmlearn.readthedocs.io/en/stable/tutorial.html#available-models
#https://www.programcreek.com/python/example/105057/hmmlearn.hmm.GaussianHMM

#Building and training the model using  hmmlearn, an HMM library for Python

def model_build(components=7, iterations=100):
  mod = [0]*7
  for i in range(len(gestures)):
    mod[i] = hmm.GaussianHMM(n_components=components, covariance_type="full", n_iter=iterations)
    mod[i].fit(np.array(x_train[i]).reshape(-1, 4), lengths=([30]*40))
  with open('models_7_HMM', 'wb') as files:
    pickle.dump(mod, files)
  model_test(mod)

In [109]:
for i in range(len(list(gestures))):
  print(list(gestures)[i])

MoveLeft
MoveRight
MoveDown
ZoomOut
Press
MoveUp
ZoomIn


In [110]:
#This is the function that we use for testing the model
accuracy=[]
def model_test(mod):
  test_size = np.array(x_test).shape[1]
  for i in range(len(gestures)):
    pred_correct=0
    print("Gesture : ",list(gestures)[i])
    for j in range(test_size):
      print("Sample ",j+1)
      m_score,m_index=0,0
      for k in range(len(mod)):
        score = mod[k].score(np.array(x_test)[i][j].reshape(30,4))
        m_score = max(m_score,score)
        if m_score == score:
          m_index = k
        print('model_num ', k+1,' model_score: ',score)
      if m_index==i:
        pred_correct+=1
    accuracy.append(pred_correct/test_size)



In [114]:
print('1 Component & 100 iterations')
model_build(1)

print('For 1 component & 100 iterations- The accuracy is')
for i in range(len(set(dataset.Gesture))):
    print('Model',i+1,'accuracy:', accuracy[i])

1 Component & 100 iterations
Gesture :  MoveLeft
Sample  1
model_num  1  model_score:  110.63819591525427
model_num  2  model_score:  140.249666610458
model_num  3  model_score:  160.64476913049927
model_num  4  model_score:  78.09233850571232
model_num  5  model_score:  139.5212312500319
model_num  6  model_score:  128.22155598496883
model_num  7  model_score:  -0.5795951595097506
Sample  2
model_num  1  model_score:  111.06832122778711
model_num  2  model_score:  141.64419438002392
model_num  3  model_score:  161.84682135429094
model_num  4  model_score:  79.96575732738637
model_num  5  model_score:  148.79111542883973
model_num  6  model_score:  129.35392366160437
model_num  7  model_score:  -0.36932997145552804
Sample  3
model_num  1  model_score:  116.55361694094091
model_num  2  model_score:  150.5026416812008
model_num  3  model_score:  171.4577997183418
model_num  4  model_score:  86.14768587991732
model_num  5  model_score:  164.84355960047674
model_num  6  model_score:  135.3

,LC_X_1,LC_X_2,LC_X_3,LC_X_4,LC_X_5,LC_X_6,LC_X_7,LC_X_8,LC_X_9,LC_X_10,...,RC_Y_21,RC_Y_22,RC_Y_23,RC_Y_24,RC_Y_25,RC_Y_26,RC_Y_27,RC_Y_28,RC_Y_29,RC_Y_30
15,0.706751,0.705263,0.704641,0.701053,0.702532,0.705263,0.705263,0.702532,0.702532,0.704641,...,0.901786,0.901786,0.910314,0.906250,0.876596,0.879828,0.854077,0.897321,0.851064,0.892857
9,0.504219,0.501053,0.502110,0.501053,0.502110,0.503158,0.505263,0.506329,0.506329,0.506329,...,0.857143,0.857143,0.860987,0.861607,0.829787,0.828326,0.828326,0.857143,0.817021,0.857143
11,0.514768,0.515789,0.516878,0.515789,0.514768,0.513684,0.511579,0.514768,0.514768,0.514768,...,0.758929,0.843750,0.852018,0.843750,0.817021,0.819742,0.824034,0.857143,0.808511,0.861607
12,0.491561,0.496842,0.500000,0.505263,0.510549,0.513684,0.515789,0.518987,0.523207,0.525316,...,0.915179,0.915179,0.923767,0.915179,0.876596,0.888412,0.884120,0.924107,0.872340,0.915179
37,0.565401,0.564211,0.565401,0.564211,0.563291,0.562105,0.562105,0.563291,0.563291,0.563291,...,0.808036,0.808036,0.811659,0.803571,0.757447,0.751073,0.751073,0.781250,0.744681,0.781250
8,0.556962,0.555789,0.559072,0.555789,0.704641,0.703158,0.555789,0.556962,0.556962,0.556962,...,0.870536,0.875000,0.883408,0.875000,0.846809,0.858369,0.875536,0.910714,0.868085,0.915179
0,0.647679,0.818947,0.546414,0.545263,0.546414,0.545263,0.545263,0.544304,0.544304,0.546414,...,0.825893,0.821429,0.825112,0.821429,0.787234,0.811159,0.815451,0.857143,0.825532,0.861607
43,0.424051,0.429474,0.428270,0.429474,0.428270,0.425263,0.421053,0.424051,0.415612,0.417722,...,0.964286,0.968750,0.977578,0.973214,0.927660,0.000000,0.000000,0.000000,0.000000,0.000000
16,0.740506,0.738947,0.740506,0.738947,0.740506,0.736842,0.736842,0.740506,0.738397,0.740506,...,0.897321,0.897321,0.905830,0.901786,0.863830,0.875536,0.879828,0.910714,0.868085,0.924107
5,0.573840,0.572632,0.571730,0.572632,0.575949,0.574737,0.574737,0.575949,0.575949,0.571730,...,0.866071,0.870536,0.878924,0.879464,0.834043,0.845494,0.849785,0.875000,0.842553,0.897321
